In [ ]:
#### 0. <a href=#Prepare-conda-environment>Prepare conda environment</a> 
#### 1. <a href=#Load-Python-Libraries>Load packages</a> 
#### 2. <a href=#Load-the-data>Load train and test files</a> 
#### 3. <a href=#Check-the-data>Check the data</a> 
#### 4. <a href=#Building-some-useful-functions>Building some useful functions</a> 
#### 5. <a href=#Feature-engineering>Feature engineering</a>
#### 5. <a href=#Prepare-the-test-set-for-submision>Prepare the test set for submision</a>
#### 7. <a href=Generate-submission-file> Generate submission file</a>

# Prepare conda environment

In [ ]:
!conda env create -n pytf36 tensorflow-gpu==1.14.0

# Load Python Libraries

In [1]:
#
# data processing
import pandas as pd
import numpy as np
from scipy.signal import medfilt
from sklearn.preprocessing import MinMaxScaler
#model estimation
from sklearn.metrics import accuracy_score
#custom functions
from config import *
from DataGenerator import *
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)


Using TensorFlow backend.
/home/geoanton/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/geoanton/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/geoanton/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/geoanton/an

/home/geoanton/anaconda3/envs/keras/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/geoanton/anaconda3/envs/keras/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/geoanton/anaconda3/envs/keras/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/geoanton/anacond

# Load the data

To run on a private test set, set `TEST_NAME` to a location of the private test csv.

In [2]:

DATA_PATH = './data/raw/'
TEST_NAME = f'{DATA_PATH}test.csv'

test = pd.read_csv(TEST_NAME)
test.head()

,row_id,well_id,GR
0,0,5000,134.943504
1,1,5000,127.004675
2,2,5000,133.159255
3,3,5000,134.411762
4,4,5000,135.748644


In [3]:
GetData=DataGenerator()

In [4]:
predictions = np.zeros((GetData.X_test.shape[0],1104,5))

for i in range(5):
    model=DL_model(input_size=(GetData.X_test.shape[1],GetData.X_test.shape[2]) ,hyperparams=HYPERPARAM)
    model.load_weights(f'./data/weights/LSTM_model_{i}_.h5')
    predictions += model.predict(GetData.X_test)/5
    
predictions = predictions[:,:1100:,:]

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1104, 70)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1104, 512)         503808    
_________________________________________________________________
activation_1 (Activation)    (None, 1104, 512)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1104, 256)         493056    
_________________________________________________________________
activation_2 (Activation)    (None, 1104, 256)         0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 1104, 256)         296448    
_________________________________________________________________
activation_3 (Activation)    (None, 1104, 256)         0   

In [5]:
predictions[0,:,0]

array([0.99823278, 0.99966702, 0.99986856, ..., 0.99996057, 0.99995205,
       0.99993415])

In [6]:
def prepare_test(pred_test, df_test):
    
    wells = df_test['well_id'].sort_values().unique().tolist()
    list_df_wells = [df_test.loc[df_test['well_id'].isin([w]), :].copy() for w in wells]
    
    for df in list_df_wells:
        df.index = np.arange(df.shape[0])

    for i, df_well in enumerate(list_df_wells):
        df_well['label'] = np.argmax(pred_test[i, :], axis=1)

    result = pd.concat(list_df_wells, axis=0)
    return result


In [7]:
submit = prepare_test(predictions, test)

In [8]:
submit[['row_id', 'well_id', 'label']].to_csv('data/result/0.985_submit.csv', index=False)